# Let's Simulate Multiple Investment Outcomes
Using the following variables, we can simulate various investment outcomes to see which long-term invesment strategies will serve us best.

### Model Variables
1. `weeklyInvestmentHypotheticals` and `monthlyInvestmentHypotheticals` (list): How much should you invest each week or month?
2. `years` (integer): Over how many years should we simulate investment growth?
3. `annualIncreaseRateHypotheticals` and `monthlyIncreaseRateHypotheticals` (list): At what rate will your annual (weekly) or monthly investment increase as the years and months go by?
4. `lowRate`: What is the lower bound of the annual or monthly interest rate you'll randomly generate each period?
5. `highRate`: What is the upper bound of the annual or monthly interest rate you'll randomly generate each period?

## Annual vs. Monthly Simulations
I've used some confusing naming conventions in this first iteration, so let me clear up a couple of things (future versions will be more clear).
 - The `investment_simulating_annual()` function simulates interest at the annual level, but the "new investment" variable is uses the term "weekly". This was done to help visualize the financial commitment at an easy-to-understand level, since $25/week is easier to understand for most folks vs. $1300/year.
 - The `investment_simulating_monthly()` function is a bit easier to understand. It simulates interest accrual monthly AND it simulates new investments monthly.
    - The monthly investment increase rate (`investmentIncreaseRateHypotheticals`) is the weird part. It's easiest to think about invesment increase rates on an annual basis (at least for me), so that's how I initially format the list of hypotheticals. Then I divide the annual increase rates by 12 to get the monthly increase rates. In this way, it's not accurate to think of the increases as annual increases, since increasing your monthly investment by "1% / 12" each month is not the same as increasing the investment 1% each year.
      - For example, I use "`investmentIncreaseRateHypotheticals` = [round(0.01/12, 4), round(0.02/12, 4), round(0.03/12, 4), round(0.04/12, 4)]". This isn't the same as a 1%, 2%, 3%, and 4% annual investment increase simulation (it works out to be more than that), but it's fairly close, and a ~1% annual investment increase is easier to digest than a 0.08% monthly increase.

## Annual Simulations (Weekly investment, annual interest)

In [1]:
from compound_interest_planner import investment_simulating_annual
import plotly.express as px

df = investment_simulating_annual(principal = 2400, 
                                  years = 50, 
                                  weeklyInvestmentHypotheticals = [25, 35, 45, 55], 
                                  annualIncreaseRateHypotheticals = [0.03, 0.04, 0.05], 
                                  lowRate =  4, 
                                  highRate = 10)

### Let's look at the output

In [2]:
df.head()

,year,annual_percent_interest,initial_weekly_investment,rate_of_annual_investment_increase,annual_investment,end_of_year_total
0,2022,0.092,25,0.03,1300.0,3920.8
1,2022,0.092,35,0.03,1820.0,4440.8
2,2022,0.092,45,0.03,2340.0,4960.8
3,2022,0.092,55,0.03,2860.0,5480.8
4,2022,0.092,25,0.04,1300.0,3920.8


In [3]:
df.describe()

,year,annual_percent_interest,initial_weekly_investment,rate_of_annual_investment_increase,annual_investment,end_of_year_total
count,600.00000,600.000000,600.000000,600.000000,600.000000,6.000000e+02
mean,2046.50000,0.067100,40.000000,0.040000,6584.053800,3.772596e+05
std,14.44291,0.017063,11.189668,0.008172,4944.639108,4.348062e+05
min,2022.00000,0.041000,25.000000,0.030000,1300.000000,3.920800e+03
25%,2034.00000,0.051000,32.500000,0.030000,3134.015000,5.447300e+04
50%,2046.50000,0.068500,40.000000,0.040000,5030.465000,1.998915e+05
75%,2059.00000,0.084000,47.500000,0.050000,8375.150000,5.662472e+05
max,2071.00000,0.096000,55.000000,0.050000,31235.060000,2.308783e+06


### Boot up some visualizations

#### How will our portfolio grow under each hypothetical plan?

In [4]:
fig1 = px.line(df, x = "year", y = "end_of_year_total", color = "initial_weekly_investment", line_dash = "rate_of_annual_investment_increase",
               title="Simulated End of Year Totals per Year")
fig1.show()

#### How much will we be investing each year under each hypothetical plan?

In [5]:
fig2 = px.line(df, x = "year", y = "annual_investment", color = "initial_weekly_investment", line_dash = "rate_of_annual_investment_increase",
               title = "Simulated Annual Investments")
fig2.show()

#### How did our random annual interest rates end up looking?

In [6]:
print(df['annual_percent_interest'].mean())

fig3 = px.line(df, x = "year", y = "annual_percent_interest",
               title = "Simulated Annual Interest Rates")
fig3.show()

0.06709999999999999


#### Getting a quick sense of ROI under each simulation: Comparing our out-of-pocket investment totals (first table) to portfolio totals at the end of the simulation (second table)

In [7]:
df.groupby(["initial_weekly_investment", "rate_of_annual_investment_increase"], as_index = False)["annual_investment"].sum().pivot("initial_weekly_investment", "rate_of_annual_investment_increase").fillna(0)

annual_investment                      
rate_of_annual_investment_increase              0.03       0.04       0.05
initial_weekly_investment                                                 
25                                         146637.39  198467.18  272153.90
35                                         205291.40  277854.99  381011.98
45                                         263944.54  357239.69  489869.06
55                                         322598.95  436626.85  598736.35

In [8]:
df.groupby(["initial_weekly_investment", "rate_of_annual_investment_increase"], as_index = False)["end_of_year_total"].max().pivot("initial_weekly_investment", "rate_of_annual_investment_increase").fillna(0)

end_of_year_total                        
rate_of_annual_investment_increase              0.03        0.04        0.05
initial_weekly_investment                                                   
25                                         765367.93   902561.29  1082905.37
35                                        1046977.43  1239055.69  1491522.60
45                                        1328585.24  1575541.37  1900137.64
55                                        1610196.03  1912033.58  2308782.66

We can look at simulation ROI with a simple bar plot, too

In [9]:
annual_table = df.groupby(["initial_weekly_investment", "rate_of_annual_investment_increase"], as_index = False)[["annual_investment","end_of_year_total"]].agg(["max","sum"]).fillna(0).reset_index()

annual_table.columns = ['_'.join(col) for col in annual_table.columns]

annual_table_melt = annual_table.melt(id_vars =['initial_weekly_investment_', 'rate_of_annual_investment_increase_'], value_vars=['annual_investment_max', 'annual_investment_sum', 'end_of_year_total_max', 'end_of_year_total_sum'])

annual_table_melt['hypotheticals'] = annual_table_melt['initial_weekly_investment_'].astype(str) + ' - ' + annual_table_melt['rate_of_annual_investment_increase_'].astype(str)

fig_annual_table1 = px.bar(annual_table_melt.loc[((annual_table_melt['variable'] == 'annual_investment_sum') | (annual_table_melt['variable'] == 'end_of_year_total_max')), ['initial_weekly_investment_', 'value', 'variable', 'hypotheticals']], 
                          x = 'hypotheticals',
                          y = 'value', color = 'variable', 
                          barmode = 'group',
                          title = "Total Investments and Corresponding Simulated Portfolio Value")
fig_annual_table1.show()

Or, looking at it another way...

In [10]:
annual_table['ROI'] = annual_table['end_of_year_total_max'] / annual_table['annual_investment_sum']
annual_table['rate_of_annual_investment_increase_'] = annual_table['rate_of_annual_investment_increase_'].astype(str)

fig_annual_table2 = px.bar(annual_table, 
                          x = 'initial_weekly_investment_',
                          y = 'ROI', color = 'rate_of_annual_investment_increase_', 
                          barmode = 'group',
                          title = "Simulated ROI")
fig_annual_table2.show()

So, in this example, each additional \$10 you start investing weekly produces a _very_ slight decrease in overall ROI, but remember that the portfolio values are much higher for each additional \$10 you spend per week:

In [11]:
annual_table['ROI'] = annual_table['end_of_year_total_max'] / annual_table['annual_investment_sum']
annual_table['rate_of_annual_investment_increase_'] = annual_table['rate_of_annual_investment_increase_'].astype(str)

fig_annual_table3 = px.bar(annual_table, 
                          x = 'initial_weekly_investment_',
                          y = 'end_of_year_total_max', color = 'rate_of_annual_investment_increase_', 
                          barmode = 'group',
                          title = "Simulated Portfolio Values")
fig_annual_table3.show()

# Monthly Simulations (Monthly investment, monthly interest rate)

In [12]:
from compound_interest_planner import investment_simulating_monthly

df_monthly = investment_simulating_monthly(principal = 2400, 
                                           years = 50,
                                           monthlyInvestmentHypotheticals = [100, 150, 200, 250], 
                                           investmentIncreaseRateHypotheticals = [round(0.01/12, 4), round(0.02/12, 4), round(0.03/12, 4), round(0.04/12, 4)],
                                           lowRate = -0.002,
                                           highRate = 0.010)

### Here's a peak at the data

In [13]:
df_monthly.head()

,year,month,date,monthly_percent_interest,initial_monthly_investment,monthly_investment_increase_rate,monthly_investment,end_of_month_total
0,2022,1,2022-01-01,0.004,100,0.0008,100.0,2509.6
1,2022,1,2022-01-01,0.004,150,0.0008,150.0,2559.6
2,2022,1,2022-01-01,0.004,200,0.0008,200.0,2609.6
3,2022,1,2022-01-01,0.004,250,0.0008,250.0,2659.6
4,2022,1,2022-01-01,0.004,100,0.0017,100.0,2509.6


### Visualization time!

Let's look at simulated portfolio values first

In [14]:
fig4 = px.line(df_monthly, x = "date", y = "end_of_month_total", color = "initial_monthly_investment", line_dash = "monthly_investment_increase_rate",
               title="Simulated End of Month Totals")
fig4.show()

#### Wait, I'll be investing __how much__ per month by 2072???

In [15]:
fig5 = px.line(df_monthly, x = "date", y = "monthly_investment", color = "initial_monthly_investment", line_dash = "monthly_investment_increase_rate",
               title = "Simulated Monthly Investments")
fig5.show()

#### Checking on the simulated interest rates; looking plenty volatile

In [16]:
fig6 = px.line(df_monthly, x = "date", y = "monthly_percent_interest",
               title = "Simulated Monthly Interest Rates")
fig6.show()

What did annual interest rates look like though?

In [17]:
import pandas as pd

print(df_monthly[['date', 'monthly_percent_interest']].set_index('date').groupby(pd.Grouper(freq='Y')).sum().mean())

fig7 = px.line(df_monthly[['date', 'monthly_percent_interest']].set_index('date').groupby(pd.Grouper(freq='Y')).sum().reset_index(), x = "date", y = "monthly_percent_interest",
               title = "Simulated Annual Interest Rates (Monthly Simulation)")
fig7.show()

monthly_percent_interest    0.74528
dtype: float64


Start looking at ROI again...

In [18]:
df_monthly.groupby(["initial_monthly_investment", "monthly_investment_increase_rate"], as_index = False)["monthly_investment"].sum().pivot("initial_monthly_investment", "monthly_investment_increase_rate").fillna(0)

monthly_investment                        \
monthly_investment_increase_rate             0.0008     0.0017     0.0025   
initial_monthly_investment                                                  
100                                        76897.64  104135.04  138900.72   
150                                       115408.25  156259.52  208411.33   
200                                       153903.41  208321.35  277864.73   
250                                       192397.97  260413.66  347350.29   

                                             
monthly_investment_increase_rate     0.0033  
initial_monthly_investment                   
100                               188469.39  
150                               282724.44  
200                               376888.50  
250                               471189.85

In [19]:
df_monthly.groupby(["initial_monthly_investment", "monthly_investment_increase_rate"], as_index = False)["end_of_month_total"].max().pivot("initial_monthly_investment", "monthly_investment_increase_rate").fillna(0)

end_of_month_total                         \
monthly_investment_increase_rate             0.0008     0.0017      0.0025   
initial_monthly_investment                                                   
100                                       299212.49  360763.24   434339.47   
150                                       436792.67  529106.25   639458.83   
200                                       574332.56  697243.88   844422.19   
250                                       711867.83  865503.61  1049447.68   

                                              
monthly_investment_increase_rate      0.0033  
initial_monthly_investment                    
100                                533396.92  
150                                787936.33  
200                               1042261.49  
250                               1296894.29

In [20]:
monthly_table = df_monthly.groupby(["initial_monthly_investment", "monthly_investment_increase_rate"], as_index = False)[["monthly_investment", "end_of_month_total"]].agg(["max","sum"]).fillna(0).reset_index()

monthly_table.columns = ['_'.join(col) for col in monthly_table.columns]

monthly_table_melt = monthly_table.melt(id_vars =['initial_monthly_investment_', 'monthly_investment_increase_rate_'], value_vars=['monthly_investment_max', 'monthly_investment_sum', 'end_of_month_total_max', 'end_of_month_total_sum'])

monthly_table_melt['hypotheticals'] = monthly_table_melt['initial_monthly_investment_'].astype(str) + ' - ' + monthly_table_melt['monthly_investment_increase_rate_'].astype(str)

fig_monthly_table = px.bar(monthly_table_melt.loc[((monthly_table_melt['variable'] == 'monthly_investment_sum') | (monthly_table_melt['variable'] == 'end_of_month_total_max')), ['initial_monthly_investment_', 'value', 'variable', 'hypotheticals']], 
                           x = 'hypotheticals',
                           y = 'value', color = 'variable', 
                           barmode = 'group')
fig_monthly_table.show()

Let's check for that _ever so slight_ diminishing return we know and love

In [21]:
monthly_table['ROI'] = monthly_table['end_of_month_total_max'] / monthly_table['monthly_investment_sum']
monthly_table['monthly_investment_increase_rate_'] = monthly_table['monthly_investment_increase_rate_'].astype(str)

fig_monthly_table2 = px.bar(monthly_table, 
                          x = 'initial_monthly_investment_',
                          y = 'ROI', color = 'monthly_investment_increase_rate_', 
                          barmode = 'group',
                          title = "Simulated ROI (Monthly Simulation)")
fig_monthly_table2.show()

Last one, I swear...

In [22]:
monthly_table['monthly_investment_increase_rate_'] = monthly_table['monthly_investment_increase_rate_'].astype(str)

fig_monthly_table3 = px.bar(monthly_table, 
                          x = 'initial_monthly_investment_',
                          y = 'end_of_month_total_max', color = 'monthly_investment_increase_rate_', 
                          barmode = 'group',
                          title = "Simulated Portfolio Values")
fig_monthly_table3.show()

## Thanks for joining me for this!

This is a blast to work on, and I hope to continue improving the tool. Overall, my goal is to create a simple tool to help people visualize the different routes they can take, while also building complex-enough simulations to provide a realistic view. I know I can add more complexity fairly easily. The harder part will be making it user-friendly for all audiences.

If you have any ideas, I'd love to hear them!

Thanks again for making it all of the way to the end.